# 1.0 Get and view data

In [1]:
import os

import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import (col, date_format, expr, from_json,
                                   from_unixtime, from_utc_timestamp, length,
                                   regexp_replace, split, sum, trim, when)
from pyspark.sql.types import StructType, StructField, StringType
from uszipcode import SearchEngine

In [2]:
data_path = os.path.join('..', 'data', 'cc_sample_transaction.json')
spark = SparkSession.builder.appName('CC Data Analysis').getOrCreate()
raw_cc_data = spark.read.json(data_path)
raw_cc_data.limit(5).show()  # Looking at the first 5 rows

+----------+------+-------------+-----------+----------------+--------+----------------+----------------------+------------------+-----------+-------------+--------------------+--------------------+---------------------+--------------------+
|Unnamed: 0|   amt|     category|     cc_bic|          cc_num|is_fraud|  merch_eff_time|merch_last_update_time|         merch_lat| merch_long|merch_zipcode|            merchant|     personal_detail|trans_date_trans_time|           trans_num|
+----------+------+-------------+-----------+----------------+--------+----------------+----------------------+------------------+-----------+-------------+--------------------+--------------------+---------------------+--------------------+
|         0|  4.97|     misc_net|CITIUS33CHI|2703186189652095|       0|1325376018798532|         1325376018666|         36.011293| -82.048315|        28705|fraud_Rippin, Kub...|{"person_name":"J...|  2019-01-01 00:00:18|0b242abb623afc578...|
|         1|107.23|  grocery_pos

In [3]:
raw_cc_data.select('personal_detail').limit(5).show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|personal_detail                                                                                                                                                                                                                                                                      |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{"person_name":"Jennifer,Banks,eeeee","gender":"F","address":"{\"street\":\"561 Perry Cove\",\"city\":\"Moravian Falls\",\"state\":\"NC\",\"zip\":\"28654\"}","

# 2.0 Data Transformation

## 2.1 Parse `personal_detail` column

In [78]:
transformed_cc_data = raw_cc_data.select('*')
address_schema = StructType([
    StructField('street', StringType(), True),
    StructField('city', StringType(), True),
    StructField('state', StringType(), True),
    StructField('zip', StringType(), True)
])

personal_details_schema = StructType([
    StructField('person_name', StringType(), True),
    StructField('gender', StringType(), True),
    StructField('address', StringType(), True),
    StructField('lat', StringType(), True),
    StructField('long', StringType(), True),
    StructField('city_pop', StringType(), True),
    StructField('job', StringType(), True),
    StructField('dob', StringType(), True)
])

transformed_cc_data = transformed_cc_data.withColumn('personal_detail_parsed',
                                                     from_json(col('personal_detail'), personal_details_schema))

transformed_cc_data = transformed_cc_data.withColumn('address_parsed',
                                                     from_json(col('personal_detail_parsed.address'), address_schema))
transformed_cc_data = transformed_cc_data.select(
    col('Unnamed: 0').alias('id'),
    col('amt').alias('amount'),
    col('is_fraud'),
    col('category'),
    col('merchant'),
    col('trans_date_trans_time'),
    col('personal_detail_parsed.person_name').alias('cc_owner'),
    col('personal_detail_parsed.gender').alias('gender'),
    col('personal_detail_parsed.job').alias('job'),
    col('personal_detail_parsed.dob').alias('date_of_birth'),
    col('address_parsed.street').alias('address_street'),
    col('address_parsed.city').alias('address_city'),
    col('address_parsed.state').alias('address_state'),
    col('address_parsed.zip').alias('address_zipcode'),
    col('personal_detail_parsed.lat').alias('cc_owner_lat'),
    col('personal_detail_parsed.long').alias('cc_owner_long'),
    col('personal_detail_parsed.city_pop').alias('city_population'),
    col('merch_eff_time'),
    col('merch_last_update_time'),
    col('merch_lat'),
    col('merch_long'),
    col('merch_zipcode'),
    col('cc_bic'),
    col('cc_num'),
    col('trans_num')
)

transformed_cc_data.limit(10).show()

+---+------+--------+-------------+--------------------+---------------------+--------------------+------+--------------------+-------------+--------------------+--------------+-------------+---------------+------------+------------------+---------------+----------------+----------------------+------------------+------------------+-------------+-----------+----------------+--------------------+
| id|amount|is_fraud|     category|            merchant|trans_date_trans_time|            cc_owner|gender|                 job|date_of_birth|      address_street|  address_city|address_state|address_zipcode|cc_owner_lat|     cc_owner_long|city_population|  merch_eff_time|merch_last_update_time|         merch_lat|        merch_long|merch_zipcode|     cc_bic|          cc_num|           trans_num|
+---+------+--------+-------------+--------------------+---------------------+--------------------+------+--------------------+-------------+--------------------+--------------+-------------+-------------

## 2.2 Casting columns

In [79]:
col_dtypes = {
    'id': 'integer',
    'amount': 'float',
    'trans_date_trans_time': 'timestamp',
    'cc_owner_lat': 'float',
    'cc_owner_long': 'float',
    'city_population': 'integer',
    'merch_eff_time': 'long',
    'merch_last_update_time': 'long',
    'merch_lat': 'float',
    'merch_long': 'float'
}

# Change multiple column types
for column in col_dtypes.keys():
    transformed_cc_data = transformed_cc_data.withColumn(column, col(column).cast(col_dtypes[column]))

transformed_cc_data.printSchema()

root
 |-- id: integer (nullable = true)
 |-- amount: float (nullable = true)
 |-- is_fraud: string (nullable = true)
 |-- category: string (nullable = true)
 |-- merchant: string (nullable = true)
 |-- trans_date_trans_time: timestamp (nullable = true)
 |-- cc_owner: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- job: string (nullable = true)
 |-- date_of_birth: string (nullable = true)
 |-- address_street: string (nullable = true)
 |-- address_city: string (nullable = true)
 |-- address_state: string (nullable = true)
 |-- address_zipcode: string (nullable = true)
 |-- cc_owner_lat: float (nullable = true)
 |-- cc_owner_long: float (nullable = true)
 |-- city_population: integer (nullable = true)
 |-- merch_eff_time: long (nullable = true)
 |-- merch_last_update_time: long (nullable = true)
 |-- merch_lat: float (nullable = true)
 |-- merch_long: float (nullable = true)
 |-- merch_zipcode: string (nullable = true)
 |-- cc_bic: string (nullable = true)
 |-- cc_num:

## 2.3 Timestamp Conversion

I'm assuming all timestamps are originally in UTC time.

### 2.3.1 `trans_date_trans_time` column

In [80]:
transformed_cc_data = transformed_cc_data.withColumn(
    'trans_date_trans_time', date_format(
        from_utc_timestamp(col('trans_date_trans_time'), 'Asia/Singapore'),
        'yyyy-MM-dd HH:mm:ss.SSSSSS XXX'))

### 2.3.2 `merch_eff_time` and `merch_last_update_time` columns

In [81]:
merch_time_data = transformed_cc_data.select(['merch_eff_time', 'merch_last_update_time'])
merch_time_data.describe().show()

+-------+--------------------+----------------------+
|summary|      merch_eff_time|merch_last_update_time|
+-------+--------------------+----------------------+
|  count|             1296675|               1296675|
|   mean|1.226971654789972...|  1.226790968939939...|
| stddev|3.678596149934444E14|   3.68057477577883E11|
|    min|         13289089638|           13253844562|
|    max|    1371816817420462|         1371816817611|
+-------+--------------------+----------------------+



`merch_eff_time` column contains 11 to 16 digit values. 16 digit values indicate the column is recoreded in microseconds.

`merch_last_update_time` column contains 11 to 13 digit values. 13 digit values indicate the column is recoreded in milliseconds.

In [82]:
transformed_cc_data = transformed_cc_data.withColumn(
    'merch_eff_time_length',
    length(col('merch_eff_time').cast('string'))
    )

transformed_cc_data = transformed_cc_data.withColumn(
    'merch_eff_time', col('merch_eff_time') / 1_000_000
    )

transformed_cc_data = transformed_cc_data.withColumn(
    'merch_last_update_time_length',
    length(col('merch_last_update_time').cast('string'))
    )

transformed_cc_data = transformed_cc_data.withColumn(
    'merch_last_update_time', col('merch_last_update_time') / 1_000
)

In [83]:
# Convert the timestamp to a human-readable format if possible
# If not, fill in the value with the other timestamp
transformed_cc_data = transformed_cc_data.withColumn(
    'merch_eff_time_new', when(
        col('merch_eff_time_length') == 16,
        date_format(
            from_utc_timestamp(
                from_unixtime(col('merch_eff_time')), 'Asia/Singapore'),
            'yyyy-MM-dd HH:mm:ss.SSSSSS XXX')
        ).otherwise(
            date_format(
                from_utc_timestamp(
                    from_unixtime(col('merch_last_update_time')), 'Asia/Singapore'),
                'yyyy-MM-dd HH:mm:ss.SSSSSS XXX'))
)

transformed_cc_data = transformed_cc_data.withColumn(
    'merch_last_update_time_new', when(
        col('merch_last_update_time_length') == 13,
        date_format(
            from_utc_timestamp(
                from_unixtime(col('merch_last_update_time')), 'Asia/Singapore'),
            'yyyy-MM-dd HH:mm:ss.SSSSSS XXX')
        ).otherwise(
            date_format(
                from_utc_timestamp(
                    from_unixtime(col('merch_eff_time')), 'Asia/Singapore'),
                'yyyy-MM-dd HH:mm:ss.SSSSSS XXX'))
)

In [84]:
transformed_cc_data = transformed_cc_data.select(
    col('id'),
    col('amount'),
    col('is_fraud'),
    col('category'),
    col('merchant'),
    col('trans_date_trans_time').alias('trans_time'),
    col('cc_owner'),
    col('gender'),
    col('job'),
    col('date_of_birth'),
    col('address_street'),
    col('address_city'),
    col('address_state'),
    col('address_zipcode'),
    col('cc_owner_lat'),
    col('cc_owner_long'),
    col('city_population'),
    col('merch_eff_time_new').alias('merch_eff_time'),
    col('merch_last_update_time_new').alias('merch_last_update_time'),
    col('merch_lat'),
    col('merch_long'),
    col('merch_zipcode'),
    col('cc_bic'),
    col('cc_num'),
    col('trans_num')
)

transformed_cc_data.limit(10).show()

+---+------+--------+-------------+--------------------+--------------------+--------------------+------+--------------------+-------------+--------------------+--------------+-------------+---------------+------------+-------------+---------------+--------------------+----------------------+---------+----------+-------------+-----------+----------------+--------------------+
| id|amount|is_fraud|     category|            merchant|          trans_time|            cc_owner|gender|                 job|date_of_birth|      address_street|  address_city|address_state|address_zipcode|cc_owner_lat|cc_owner_long|city_population|      merch_eff_time|merch_last_update_time|merch_lat|merch_long|merch_zipcode|     cc_bic|          cc_num|           trans_num|
+---+------+--------+-------------+--------------------+--------------------+--------------------+------+--------------------+-------------+--------------------+--------------+-------------+---------------+------------+-------------+---------

## 2.4 Extracting first and last names

In [85]:
transformed_cc_data.select('cc_owner').limit(20).show(truncate=False)

+-------------------------+
|cc_owner                 |
+-------------------------+
|Jennifer,Banks,eeeee     |
|Stephanie,Gill,eeeee     |
|Edward@Sanchez           |
|Jeremy/White, !          |
|Tyler@Garcia             |
|Jennifer,Conner,eeeee    |
|Kelsey, , Richards NOOOO |
|Steven, Williams         |
|Heather, , Chase NOOOO   |
|Melissa@Aguilar          |
|Eddie|Mendez!!!          |
|Theresa@Blackwell        |
|Charles|Robles!!!        |
|Jack@Hill                |
|Christopher@Castaneda    |
|Ronald@Carson            |
|Lisa, Mendez             |
|Nathan,Thomas,eeeee      |
|Justin, Gay              |
|Kenneth, , Robinson NOOOO|
+-------------------------+



In [86]:
# Removing meaningless words
transformed_cc_data = transformed_cc_data.withColumn(
    'cc_owner_new',
    regexp_replace(col('cc_owner'), 'NOOOO', '')
)

transformed_cc_data = transformed_cc_data.withColumn(
    'cc_owner_new',
    regexp_replace(col('cc_owner_new'), 'eeeee', '')
)

# Removing trailing whitespaces
transformed_cc_data = transformed_cc_data.withColumn(
    'cc_owner_new',
    trim(col('cc_owner_new'))
)

# Replacing non-alphabetic and comma characters with commas
transformed_cc_data = transformed_cc_data.withColumn(
    'cc_owner_new',
    regexp_replace(col('cc_owner_new'), r'[^a-zA-Z,]', ',')
)
# Splitting the names based on commas
transformed_cc_data = transformed_cc_data.withColumn(
    'cc_owner_new', split(col('cc_owner_new'), ','))

# Filter out empty strings from the names array
transformed_cc_data = transformed_cc_data.withColumn(
    'cc_owner_new',
    expr("filter(cc_owner_new, x -> x != '')")
)

transformed_cc_data = transformed_cc_data.withColumn(
    'cc_owner_first_name', col('cc_owner_new')[0]).withColumn(
        'cc_owner_last_name', col('cc_owner_new')[1])

transformed_cc_data.select(['cc_owner_new', 'cc_owner_first_name',
                            'cc_owner_last_name']).limit(20).show(truncate=False)

+------------------------+-------------------+------------------+
|cc_owner_new            |cc_owner_first_name|cc_owner_last_name|
+------------------------+-------------------+------------------+
|[Jennifer, Banks]       |Jennifer           |Banks             |
|[Stephanie, Gill]       |Stephanie          |Gill              |
|[Edward, Sanchez]       |Edward             |Sanchez           |
|[Jeremy, White]         |Jeremy             |White             |
|[Tyler, Garcia]         |Tyler              |Garcia            |
|[Jennifer, Conner]      |Jennifer           |Conner            |
|[Kelsey, Richards]      |Kelsey             |Richards          |
|[Steven, Williams]      |Steven             |Williams          |
|[Heather, Chase]        |Heather            |Chase             |
|[Melissa, Aguilar]      |Melissa            |Aguilar           |
|[Eddie, Mendez]         |Eddie              |Mendez            |
|[Theresa, Blackwell]    |Theresa            |Blackwell         |
|[Charles,

In [87]:
transformed_cc_data = transformed_cc_data.select(
    col('id'),
    col('amount'),
    col('is_fraud'),
    col('category'),
    col('merchant'),
    col('trans_time'),
    col('cc_owner_first_name'),
    col('cc_owner_last_name'),
    col('gender'),
    col('job'),
    col('date_of_birth'),
    col('address_street'),
    col('address_city'),
    col('address_state'),
    col('address_zipcode'),
    col('cc_owner_lat'),
    col('cc_owner_long'),
    col('city_population'),
    col('merch_eff_time'),
    col('merch_last_update_time'),
    col('merch_lat'),
    col('merch_long'),
    col('merch_zipcode'),
    col('cc_bic'),
    col('cc_num'),
    col('trans_num')
)

## 2.5 Cleansing `merchant` and `cc_bic` columns

In [88]:
# Removing meaningless words
transformed_cc_data = transformed_cc_data.withColumn(
    'merchant_new',
    regexp_replace(col('merchant'), 'fraud_', '')
)

# Converting null values to None
transformed_cc_data = transformed_cc_data.withColumn(
    'cc_bic_new', when((col('cc_bic') == 'Null') |
                       (col('cc_bic') == '') |
                       (col('cc_bic') == 'NA'), None).otherwise(
                           col('cc_bic')))

transformed_cc_data_final = transformed_cc_data.select(
    col('id'),
    col('amount'),
    col('is_fraud'),
    col('category'),
    col('merchant'),
    col('trans_time'),
    col('cc_owner_first_name'),
    col('cc_owner_last_name'),
    col('gender'),
    col('job'),
    col('date_of_birth'),
    col('address_street'),
    col('address_city'),
    col('address_state'),
    col('address_zipcode'),
    col('cc_owner_lat'),
    col('cc_owner_long'),
    col('city_population'),
    col('merch_eff_time'),
    col('merch_last_update_time'),
    col('merch_lat'),
    col('merch_long'),
    col('merch_zipcode'),
    col('cc_bic_new').alias('cc_bic'),
    col('cc_num'),
    col('trans_num')
)

transformed_cc_data_final.limit(10).show()

+---+------+--------+-------------+--------------------+--------------------+-------------------+------------------+------+--------------------+-------------+--------------------+--------------+-------------+---------------+------------+-------------+---------------+--------------------+----------------------+---------+----------+-------------+-----------+----------------+--------------------+
| id|amount|is_fraud|     category|            merchant|          trans_time|cc_owner_first_name|cc_owner_last_name|gender|                 job|date_of_birth|      address_street|  address_city|address_state|address_zipcode|cc_owner_lat|cc_owner_long|city_population|      merch_eff_time|merch_last_update_time|merch_lat|merch_long|merch_zipcode|     cc_bic|          cc_num|           trans_num|
+---+------+--------+-------------+--------------------+--------------------+-------------------+------------------+------+--------------------+-------------+--------------------+--------------+------------

## 3.0 Addressing missing/NULL values

In [90]:
null_counts = transformed_cc_data_final.select([
    sum((col(column).isNull()).cast('int')).alias(column)
    for column in transformed_cc_data_final.columns
])
null_counts.show()

+---+------+--------+--------+--------+----------+-------------------+------------------+------+---+-------------+--------------+------------+-------------+---------------+------------+-------------+---------------+--------------+----------------------+---------+----------+-------------+------+------+---------+
| id|amount|is_fraud|category|merchant|trans_time|cc_owner_first_name|cc_owner_last_name|gender|job|date_of_birth|address_street|address_city|address_state|address_zipcode|cc_owner_lat|cc_owner_long|city_population|merch_eff_time|merch_last_update_time|merch_lat|merch_long|merch_zipcode|cc_bic|cc_num|trans_num|
+---+------+--------+--------+--------+----------+-------------------+------------------+------+---+-------------+--------------+------------+-------------+---------------+------------+-------------+---------------+--------------+----------------------+---------+----------+-------------+------+------+---------+
|  0|     0|       0|       0|       0|         0|           

The `cc_bic` column contains null values. However, I do not believe there is a way to find this information from the data.

The `merch_zipcode` column also contains null values and they can be filled in using the `merch_lat` and `merch_long` columns.

## 3.1.1 Reducing missing values in `merch_zipcode` column

In [ ]:
# Verifying the results of uszipcode library over a sample data
merch_zipcode_sample = transformed_cc_data_final.filter(col('merch_zipcode').isNotNull()).limit(5)
merch_zipcode_sample = merch_zipcode_sample.select(['merch_lat', 'merch_long', 'merch_zipcode'])
merch_zipcode_sample = merch_zipcode_sample.toPandas()

search = SearchEngine()
def get_zipcode(lat, long):
    result = search.by_coordinates(lat=lat, lng=long)
    if result:
        return result[0].zipcode
    else:
        return None

merch_zipcode_sample['calculated_zipcode'] = merch_zipcode_sample.apply(
    lambda x: get_zipcode(x.merch_lat, x.merch_long), axis=1)
merch_zipcode_sample


,merch_lat,merch_long,merch_zipcode,calculated_zipcode
0,36.011292,-82.048317,28705,28705
1,43.150703,-112.154480,83236,83236
2,38.674999,-78.632462,22844,22844
3,40.653381,-76.152664,17972,17972
4,38.948090,-78.540298,22644,22644


The zipcodes match exactly over the sample dataset; the library is reliable and we can run it for the rows with missing values.

In [ ]:
# missing_merch_zipcode = transformed_cc_data_final.filter(col('merch_zipcode').isNull())
# missing_merch_zipcode = missing_merch_zipcode.select(['merch_lat', 'merch_long', 'merch_zipcode'])
# missing_merch_zipcode = missing_merch_zipcode.toPandas()
# missing_merch_zipcode['calculated_zipcode'] = missing_merch_zipcode.apply(
#     lambda x: get_zipcode(x.merch_lat, x.merch_long), axis=1)
# # Saving results to save time in the future
# missing_merch_zipcode[['merch_lat', 'merch_long', 'calculated_zipcode']].to_parquet(
#     os.path.join('..', 'data', 'calculated_zipcodes.parquet'), index=False)
# missing_merch_zipcode.head()

In [92]:
missing_merch_zipcode = spark.read.parquet(os.path.join('..', 'data', 'calculated_zipcodes.parquet'))
missing_merch_zipcode = missing_merch_zipcode.withColumnRenamed('merch_lat', 'merch_lat_todrop')
missing_merch_zipcode = missing_merch_zipcode.withColumnRenamed('merch_long', 'merch_long_todrop')
cleansed_cc_data = transformed_cc_data_final.join(
    missing_merch_zipcode,
    (transformed_cc_data_final['merch_lat'] == missing_merch_zipcode['merch_lat_todrop']) &
    (transformed_cc_data_final['merch_long'] == missing_merch_zipcode['merch_long_todrop']),
    how='left')
cleansed_cc_data = cleansed_cc_data.withColumn(
    'merch_zipcode',
    when(
        col('merch_zipcode').isNull(), col('calculated_zipcode')
        ).otherwise(col('merch_zipcode'))
)

for column in missing_merch_zipcode.columns:
    cleansed_cc_data = cleansed_cc_data.drop(column)

In [94]:
null_counts = cleansed_cc_data.select(
    sum(
        (col('merch_zipcode').isNull()).cast('int')
        ).alias('merch_zipcode'))
null_counts.show()

+-------------+
|merch_zipcode|
+-------------+
|        59882|
+-------------+



We've managed to reduced the missing data in `merch_zipcode` from 195,973 (15%) to 59,882 (5%) rows.

# 4.0 Analysing the data

1. Initially, all columns are strings.
2. There are 1,296,675 transactions.
3. Data spans January 1, 2019 to June 21, 2020.
4. The data contains null values.
5. Most transactions relate to travel, least transactions relate to entertainment.
6. There are also fraud cases in the data.

In [59]:
from pyspark.sql.functions import count

# Group by 'merchant' and 'merch_eff_time', then count the rows in each group
grouped_df = cc_data.groupBy("merchant", "merch_last_update_time").agg(count("*").alias("row_count"))

# Show the grouped DataFrame
grouped_df.show()

+--------------------+----------------------+---------+
|            merchant|merch_last_update_time|row_count|
+--------------------+----------------------+---------+
|fraud_Reichert, H...|         1325379419461|        1|
|fraud_Streich, Ha...|           13253955921|        1|
|fraud_Adams, Kova...|         1325407009293|        1|
|    fraud_Barton Inc|         1325410422100|        1|
|fraud_Schumm, Bau...|         1325410956149|        1|
|fraud_Heidenreich...|         1325414448242|        1|
|   fraud_Lockman Ltd|         1325415238378|        1|
|fraud_Goyette, Ho...|         1325422035766|        1|
|fraud_Swaniawski,...|         1325423065476|        1|
|  fraud_Schmeler Inc|         1325435865977|        1|
|fraud_Kutch-Wilde...|         1325437432727|        1|
|fraud_Wuckert, Wi...|          132545036954|        1|
|fraud_Bernier and...|         1325452448770|        1|
|fraud_Lubowitz-Wa...|         1325454239320|        1|
|fraud_Stiedemann Ltd|         1325456901399|   

In [23]:
cc_data.groupby('is_fraud').count().show()

+--------+-------+
|is_fraud|  count|
+--------+-------+
|       0|1289169|
|       1|   7506|
+--------+-------+



Around 0.5% of the transactions are fraud. This should be considered if training a ML model.